In [126]:
import pandas as pd
import numpy
import requests
import json

import csv

In [127]:
#extracting of data

url = "https://realty-mole-property-api.p.rapidapi.com/randomProperties"

querystring = {"limit":"100000"}

headers = {
	"x-rapidapi-key": "ce3693686amshd8a81ba62ce435ep1113afjsn137d2b9aa172",
	"x-rapidapi-host": "realty-mole-property-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

data =response.json()


filename = 'property.json'


with open(filename, 'w') as file:
    json.dump(data, file)


In [128]:

property_df = pd.read_json('property.json')

property_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   addressLine1      500 non-null    object 
 1   city              500 non-null    object 
 2   state             500 non-null    object 
 3   zipCode           500 non-null    int64  
 4   formattedAddress  500 non-null    object 
 5   assessorID        369 non-null    object 
 6   bedrooms          342 non-null    float64
 7   county            500 non-null    object 
 8   legalDescription  363 non-null    object 
 9   squareFootage     391 non-null    float64
 10  subdivision       305 non-null    object 
 11  yearBuilt         376 non-null    float64
 12  zoning            203 non-null    object 
 13  bathrooms         372 non-null    float64
 14  lotSize           363 non-null    float64
 15  propertyType      417 non-null    object 
 16  features          459 non-null    object 
 1

In [129]:
property_df['propertyTaxes']

0                              {'2019': {'total': 4207}}
1                              {'2022': {'total': 1555}}
2      {'2022': {'total': 4051}, '2023': {'total': 44...
3      {'2020': {'total': 2398}, '2022': {'total': 24...
4                             {'2021': {'total': 12711}}
                             ...                        
495                            {'2022': {'total': 5487}}
496    {'2022': {'total': 7130}, '2023': {'total': 72...
497                            {'2023': {'total': 3306}}
498                            {'2022': {'total': 3160}}
499                            {'2022': {'total': 7995}}
Name: propertyTaxes, Length: 500, dtype: object

# Transforming the dataset

In [131]:
# Extract taxAssessment2021_Values
property_df['taxAssessment2021_Values'] = property_df['taxAssessment'].apply(
    lambda x: x.get('2021', {}).get('value') if isinstance(x, dict) else None
)

# Extract propertyTaxes2021_Total
property_df['propertyTaxes2021_Total'] = property_df['propertyTaxes'].apply(
    lambda x: x.get('2021', {}).get('total') if isinstance(x, dict) else None
)

# Extract taxAssessment2022_Values
property_df['taxAssessment2022_Values'] = property_df['taxAssessment'].apply(
    lambda x: x.get('2022', {}).get('value') if isinstance(x, dict) else None
)

# Extract propertyTaxes2022_Total
property_df['propertyTaxes2022_Total'] = property_df['propertyTaxes'].apply(
    lambda x: x.get('2022', {}).get('total') if isinstance(x, dict) else None
)

# Extract taxAssessment2023_Values
property_df['taxAssessment2023_Values'] = property_df['taxAssessment'].apply(
    lambda x: x.get('2023', {}).get('value') if isinstance(x, dict) else None
)

# Extract propertyTaxes2023_Total
property_df['propertyTaxes2023_Total'] = property_df['propertyTaxes'].apply(
    lambda x: x.get('2023', {}).get('total') if isinstance(x, dict) else None
)


In [132]:
# approach of cleaning
property_df['ownerName'] = property_df['owner'].apply(lambda x: x['names'][0] if isinstance(x, dict) and isinstance(x['names'], list) and x['names'] else None)


property_df['features'] = property_df['features'].apply(json.dumps)

# second approach to find and replace the nulls
property_df.fillna({
        'bedrooms': 0,
        'ownerNames':  'Unknown',
        'addressLine2': 'Not available', 
        'squareFootage': 0,
        'yearBuilt': 0,
        'features': 'None',
        'assessorID': 'Unknown',
        'legalDescription': 'Not available',
        'subdivision': 'Not available', 
         'zoning': 'Unknown', 
         'bathrooms': 0, 
         'lotSize': 0,
         'propertyType': 'Unknown', 
         'taxAssessment': 'Not available',
        'propertyTaxes':  'Not available', 
         'lastSalePrice': 0,
        'lastSaleDate': 0,
        'ownerOccupied': 0,
        'addressLine2': 'Unknown',
        'county': 'Not available'}, inplace =True)

# to create the id from the address
property_df['id'] = property_df['id'].apply(lambda x: id(x))

property_df['lastSaleDate'] = pd.to_datetime(property_df['lastSaleDate'], format="%Y-%m-%dT%H:%M:%S.%f%z", errors='coerce')
property_df= property_df.dropna(subset=['lastSaleDate'])


In [133]:
import datetime as dt

property_df['year'] = property_df['lastSaleDate'].dt.year
property_df['month'] = property_df['lastSaleDate'].dt.month
property_df['monthName'] = property_df['lastSaleDate'].dt.month_name()
property_df['quarter'] = property_df['lastSaleDate'].dt.quarter
property_df

,addressLine1,city,state,zipCode,formattedAddress,assessorID,bedrooms,county,legalDescription,squareFootage,...,propertyTaxes2021_Total,taxAssessment2022_Values,propertyTaxes2022_Total,taxAssessment2023_Values,propertyTaxes2023_Total,ownerName,year,month,monthName,quarter
1,504 Patterson Ave,Hampton,VA,23669,"504 Patterson Ave, Hampton, VA 23669",02D020 00 00012,2.0,Hampton City,RANSONE BROS.PEMBROKE L12.,1042.0,...,NaN,120800.0,1555.0,131600.0,NaN,Bankim Patel,2018,6,June,2
2,13036 Baybrook Ln,Clermont,FL,34711,"13036 Baybrook Ln, Clermont, FL 34711",32-22-26-2000-000-07300,3.0,Lake,BENT TREE PHASE I PB 51 PG 6-7 LOT 73 ORB 5075...,2120.0,...,NaN,281061.0,4051.0,281061.0,4402.0,Arjun Vyas,2018,3,March,1
5,30 Irving St,Jersey City,NJ,7307,"30 Irving St, Jersey City, NJ 07307",06 00803-0000-00011,8.0,Hudson,LOT:11 BLK:803 DIST:06 CITY/MUNI/TWP:JERSEY CI...,2448.0,...,NaN,NaN,12566.0,593300.0,NaN,Luis Cuzco,2003,2,February,1
6,1212 Baldridge Dr,Gatesville,TX,76528,"1212 Baldridge Dr, Gatesville, TX 76528",1-08-89-9380,0.0,Coryell,"WESTERN OAKS, BLOCK 1, LOT 20, ACRES .2273",1207.0,...,NaN,163860.0,2567.0,NaN,NaN,GINA REED,2010,11,November,4
8,8231 Basswood Way,Citrus Heights,CA,95621,"8231 Basswood Way, Citrus Heights, CA 95621",209-0145-011-0000,3.0,Sacramento,"GRAND OAKS 06, LOT 182",1002.0,...,NaN,NaN,NaN,141605.0,5306.0,Sarah E Gould,2010,12,December,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1650 Palo Duro Dr,Odessa,TX,79762,"1650 Palo Duro Dr, Odessa, TX 79762",082350047400000,0.0,Ector,DESERT RIDGE 2ND FILING BLOCK 15 LOT 5 PHASE 3,2318.0,...,NaN,338677.0,5487.0,NaN,NaN,VEGA ANTHONY,2020,5,May,2
496,957 Iron Horse Dr,San Marcos,CA,92078,"957 Iron Horse Dr, San Marcos, CA 92078",221-721-02-00,4.0,San Diego,TR 12945 LOT 32,1746.0,...,NaN,603633.0,7130.0,615705.0,7228.0,Cameron Rogers,2015,4,April,2
497,4508 Alexandria Ln SE,Lacey,WA,98503,"4508 Alexandria Ln SE, Lacey, WA 98503",5905-00-05900,2.0,Thurston,SECTION 32 TOWNSHIP 18 RANGE 1W QUARTER SW NE ...,1231.0,...,NaN,345500.0,NaN,NaN,3306.0,Kristine M Thurman,2021,11,November,4
498,111 Lena Ln,Kyle,TX,78640,"111 Lena Ln, Kyle, TX 78640",R164359,0.0,Hays,"GRIST MILL HIGHLANDS SEC 1, BLOCK C, LOT 23",1334.0,...,NaN,154140.0,3160.0,276270.0,NaN,Kira Leone Wright,2022,5,May,2


In [134]:
features_dim = property_df[['features', 'propertyType', 'zoning']].drop_duplicates().reset_index(drop=True)
features_dim['features_id'] = features_dim.index +1
property_df = property_df.merge(
    features_dim[['features_id','features', 'propertyType', 'zoning']],  # Bring sales_id into propertyrecords_df
    on=['features', 'propertyType', 'zoning'],  # Match on shared columns
    how='left'
)

In [135]:
legal_dim = property_df[['legalDescription', 'subdivision']].drop_duplicates().reset_index(drop=True)
legal_dim['legal_id'] = legal_dim.index +1
property_df = property_df.merge(
    legal_dim[['legal_id','legalDescription', 'subdivision']],  # Bring sales_id into propertyrecords_df
    on=['legalDescription', 'subdivision'],  # Match on shared columns
    how='left'
)

legal_dim

,legalDescription,subdivision,legal_id
0,RANSONE BROS.PEMBROKE L12.,RANSONE BROS.PEMBROKE,1
1,BENT TREE PHASE I PB 51 PG 6-7 LOT 73 ORB 5075...,BENT TREE PHASE I,2
2,LOT:11 BLK:803 DIST:06 CITY/MUNI/TWP:JERSEY CI...,THE HERITAGE HOMESTEAD,3
3,"WESTERN OAKS, BLOCK 1, LOT 20, ACRES .2273",WESTERN OAKS,4
4,"GRAND OAKS 06, LOT 182",GRAND OAKS 06,5
...,...,...,...
289,DESERT RIDGE 2ND FILING BLOCK 15 LOT 5 PHASE 3,DESERT RIDGE 2ND FILING,290
290,TR 12945 LOT 32,SAN MARCOS TRACT #366 UNIT #2,291
291,SECTION 32 TOWNSHIP 18 RANGE 1W QUARTER SW NE ...,LEXINGTON,292
292,"GRIST MILL HIGHLANDS SEC 1, BLOCK C, LOT 23",GRIST MILL HIGHLANDS SEC 1,293


In [136]:
location_dim = property_df[['county','zipCode','formattedAddress','state','city']].drop_duplicates().reset_index(drop=True)
location_dim['location_id'] = location_dim.index +1

property_df = property_df.merge(
    location_dim[['location_id','county','zipCode','formattedAddress','state','city']],  # Bring sales_id into propertyrecords_df
    on=['county','zipCode','formattedAddress','state','city'],  # Match on shared columns
    how='left'
)


In [137]:

location_dim

,county,zipCode,formattedAddress,state,city,location_id
0,Hampton City,23669,"504 Patterson Ave, Hampton, VA 23669",VA,Hampton,1
1,Lake,34711,"13036 Baybrook Ln, Clermont, FL 34711",FL,Clermont,2
2,Hudson,7307,"30 Irving St, Jersey City, NJ 07307",NJ,Jersey City,3
3,Coryell,76528,"1212 Baldridge Dr, Gatesville, TX 76528",TX,Gatesville,4
4,Sacramento,95621,"8231 Basswood Way, Citrus Heights, CA 95621",CA,Citrus Heights,5
...,...,...,...,...,...,...
303,Ector,79762,"1650 Palo Duro Dr, Odessa, TX 79762",TX,Odessa,304
304,San Diego,92078,"957 Iron Horse Dr, San Marcos, CA 92078",CA,San Marcos,305
305,Thurston,98503,"4508 Alexandria Ln SE, Lacey, WA 98503",WA,Lacey,306
306,Hays,78640,"111 Lena Ln, Kyle, TX 78640",TX,Kyle,307


In [138]:
date_dim = property_df[['lastSaleDate', 'year', 'month',	'monthName', 'quarter']].drop_duplicates().reset_index(drop=True)
date_dim['date_id'] = date_dim.index +1

property_df = property_df.merge(
    date_dim[['date_id','lastSaleDate', 'year', 'month',	'monthName', 'quarter']],  # Bring sales_id into propertyrecords_df
    on=['lastSaleDate', 'year', 'month',	'monthName', 'quarter'],  # Match on shared columns
    how='left'
)
date_dim

,lastSaleDate,year,month,monthName,quarter,date_id
0,2018-06-27 00:00:00+00:00,2018,6,June,2,1
1,2018-03-05 00:00:00+00:00,2018,3,March,1,2
2,2003-02-19 00:00:00+00:00,2003,2,February,1,3
3,2010-11-23 00:00:00+00:00,2010,11,November,4,4
4,2010-12-03 00:00:00+00:00,2010,12,December,4,5
...,...,...,...,...,...,...
288,2020-05-26 00:00:00+00:00,2020,5,May,2,289
289,2015-04-02 00:00:00+00:00,2015,4,April,2,290
290,2021-11-24 00:00:00+00:00,2021,11,November,4,291
291,2022-05-03 00:00:00+00:00,2022,5,May,2,292


In [139]:
owner_dim = property_df[['ownerName','ownerOccupied']].drop_duplicates().reset_index(drop=True)
owner_dim['owner_id'] = owner_dim.index +1
# Merge features_dim into propertyrecords_df
property_df = property_df.merge(
    owner_dim[['owner_id','ownerName','ownerOccupied']],  # Use 'feature_id' consistently
    on=['ownerName','ownerOccupied'],  # Match on shared columns
    how='left'
)

In [140]:
property_df.columns

Index(['addressLine1', 'city', 'state', 'zipCode', 'formattedAddress',
       'assessorID', 'bedrooms', 'county', 'legalDescription', 'squareFootage',
       'subdivision', 'yearBuilt', 'zoning', 'bathrooms', 'lotSize',
       'propertyType', 'features', 'taxAssessment', 'propertyTaxes', 'owner',
       'id', 'longitude', 'latitude', 'lastSalePrice', 'lastSaleDate',
       'ownerOccupied', 'addressLine2', 'taxAssessment2021_Values',
       'propertyTaxes2021_Total', 'taxAssessment2022_Values',
       'propertyTaxes2022_Total', 'taxAssessment2023_Values',
       'propertyTaxes2023_Total', 'ownerName', 'year', 'month', 'monthName',
       'quarter', 'features_id', 'legal_id', 'location_id', 'date_id',
       'owner_id'],
      dtype='object')

In [144]:
property_df['date_id']

0        1
1        2
2        3
3        4
4        5
      ... 
303    289
304    290
305    291
306    292
307    293
Name: date_id, Length: 308, dtype: int64

In [146]:
fact_table = property_df[['id', 'date_id','owner_id', 'features_id', 'location_id', 'bedrooms', 'squareFootage', 'taxAssessment2021_Values','propertyTaxes2021_Total',
                'taxAssessment2022_Values','propertyTaxes2022_Total', 'taxAssessment2023_Values', 'propertyTaxes2023_Total','bathrooms', 'lotSize', 'lastSalePrice', 'longitude', 'latitude']]

fact_table = pd.DataFrame(fact_table)
fact_table.to_csv('property_fact.csv', index=False)
fact_table

,id,date_id,owner_id,features_id,location_id,bedrooms,squareFootage,taxAssessment2021_Values,propertyTaxes2021_Total,taxAssessment2022_Values,propertyTaxes2022_Total,taxAssessment2023_Values,propertyTaxes2023_Total,bathrooms,lotSize,lastSalePrice,longitude,latitude
0,4941869856,1,1,1,1,2.0,1042.0,NaN,NaN,120800.0,1555.0,131600.0,NaN,1.0,3001.0,95000.0,-76.360658,37.032631
1,4941870256,2,2,2,2,3.0,2120.0,235017.0,NaN,281061.0,4051.0,281061.0,4402.0,3.0,11912.0,265000.0,-81.737277,28.522662
2,4945580544,3,3,3,3,8.0,2448.0,NaN,NaN,NaN,12566.0,593300.0,NaN,0.0,2600.0,275000.0,-74.045631,40.753725
3,4494684960,4,4,4,4,0.0,1207.0,NaN,NaN,163860.0,2567.0,NaN,NaN,2.0,87120.0,0.0,-97.772332,31.435739
4,4980293968,5,5,5,5,3.0,1002.0,NaN,NaN,NaN,NaN,141605.0,5306.0,1.0,6098.0,115000.0,-121.306190,38.715607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,5057804080,289,291,284,304,0.0,2318.0,NaN,NaN,338677.0,5487.0,NaN,NaN,0.0,6782.0,0.0,-102.369031,31.903031
304,5057804560,290,292,257,305,4.0,1746.0,NaN,NaN,603633.0,7130.0,615705.0,7228.0,2.5,4544.0,533000.0,-117.175376,33.124335
305,5057804960,291,293,285,306,2.0,1231.0,NaN,NaN,345500.0,NaN,NaN,3306.0,1.5,4356.0,420000.0,-122.830378,47.006597
306,5057805440,292,294,286,307,0.0,1334.0,NaN,NaN,154140.0,3160.0,276270.0,NaN,0.0,0.0,0.0,-97.817569,29.964041


In [148]:
# converting list to a DataFrame

owner_dim = pd.DataFrame(owner_dim)
legal_dim = pd.DataFrame(legal_dim)
location_dim = pd.DataFrame(location_dim)
date_dim = pd.DataFrame(date_dim)
features_dim = pd.DataFrame(features_dim)

In [150]:
# saving dataset as csv

owner_dim.to_csv('owners_dimension.csv', index=False)
legal_dim.to_csv('legal_dimension.csv', index=False)
location_dim.to_csv('location_dimension.csv', index = False)
date_dim.to_csv('date_dimension.csv', index = False)
features_dim.to_csv('features_dimension.csv', index = False)

# Loading Data to Postgres

In [153]:
import psycopg2
import getpass

In [155]:
password = getpass.getpass()
print('password secured')

 ········


password secured


In [157]:
def get_db_connection():
    try:
        connection = psycopg2.connect(
            host='localhost',
            port=5432,
            user='postgres',
            password=password,
            database='zipco_db'
        )
        print("Database connection successful!")
        return connection
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    return None
    
get_db_connection()

Database connection successful!


<connection object at 0x12d5e2b90; dsn: 'user=postgres password=xxx dbname=zipco_db host=localhost port=5432', closed: 0>

In [159]:
def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()

    # Separate the schema creation
    create_schema_query = '''
        CREATE SCHEMA IF NOT EXISTS zipco;
    '''

    # Create tables query
    create_table_query = '''
        DROP TABLE IF EXISTS zipco.fact_table CASCADE;
        DROP TABLE IF EXISTS zipco.location_dim CASCADE;
        DROP TABLE IF EXISTS zipco.owners_dim CASCADE;
        DROP TABLE IF EXISTS zipco.legal_dim CASCADE;
        DROP TABLE IF EXISTS zipco.date_dim CASCADE;
        DROP TABLE IF EXISTS zipco.features_dim CASCADE;

        CREATE TABLE zipco.location_dim (
            county VARCHAR(300),
            zipCode INTEGER,
            formattedAddress TEXT,
            state VARCHAR(200),
            city VARCHAR(200),
            location_id INT PRIMARY KEY
        );

        CREATE TABLE zipco.date_dim (
            lastSaleDate DATE,
            year INTEGER,
            month INTEGER,
            monthName TEXT,
            quarter INTEGER,
            date_id INT PRIMARY KEY
        );

        CREATE TABLE zipco.features_dim (
            features TEXT, 
            propertyType TEXT,
            zoning TEXT,
            features_id INT PRIMARY KEY
        );

        CREATE TABLE zipco.legal_dim (
            legalDescription TEXT,
            subdivision TEXT,
            legal_id INT PRIMARY KEY
        );

        CREATE TABLE zipco.owners_dim (
            ownerName TEXT, 
            ownerOccupied FLOAT,
            owner_id INT PRIMARY KEY
        );

        CREATE TABLE zipco.fact_table (
            id BIGINT PRIMARY KEY, 
            date_id INT REFERENCES zipco.date_dim(date_id) ON DELETE CASCADE,
            features_id INT REFERENCES zipco.features_dim(features_id) ON DELETE CASCADE,
            location_id INT REFERENCES zipco.location_dim(location_id) ON DELETE CASCADE,
            owner_id INT REFERENCES zipco.owners_dim(owner_id) ON DELETE CASCADE,
            legal_id INT REFERENCES zipco.legal_dim(legal_id) ON DELETE CASCADE,
            bedrooms FLOAT,
            squareFootage FLOAT,
            bathrooms FLOAT, 
            lotSize FLOAT,
            lastSalePrice FLOAT,
            taxAssessment2021_Values FLOAT,
            propertyTaxes2021_Total FLOAT,
            taxAssessment2022_Values FLOAT, 
            propertyTaxes2022_Total FLOAT,
            taxAssessment2023_Values FLOAT, 
            propertyTaxes2023_Total FLOAT, 
            longitude FLOAT, 
            latitude FLOAT
        );
    '''

    try:
        print("Creating schema...")
        cursor.execute(create_schema_query)
        
        print("Creating tables...")
        cursor.execute(create_table_query)
        conn.commit()
        print("Tables created successfully.")
    except Exception as e:
        print(f"Error occurred during table creation: {e}")
    finally:
        cursor.close()
        conn.close()
        print("Database connection closed.")

# Run the function
create_tables()

Database connection successful!
Creating schema...
Creating tables...
Tables created successfully.
Database connection closed.


In [179]:
def load_data(csv_path, table_name, column_name):
    """
    Loading the CSV file into a table in the database.

    Args:
        csv_path (str): The path to the CSV file containing the data.
        table_name (str): The name of the table to load the data into.
        column_name (list[str]): The names of the columns in the table, in the order they appear in the CSV file.

    Returns:
        None
    """
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row
        for row in reader:
            # Replace 'Not available' or empty string with None for null values
            row = [None if cell in ('', 'Not available', ' ') else cell for cell in row]
            
            # Prepare placeholders for the insert statement
            placeholders = ', '.join(['%s'] * len(row))
            query = f'INSERT INTO {table_name} ({", ".join(column_name)}) VALUES ({placeholders});'
            
            # Execute the query with the row data
            cursor.execute(query, row)

    # Commit the changes to the database
    conn.commit()
    cursor.close()
    conn.close()
    print(f"Data loaded successfully into {table_name}.")

# #  Load data for the owner table
#owner_csv_path = 'owners_dimension.csv'
#load_data(owner_csv_path, 'zipco.owners_dim', ['ownerName', 'ownerOccupied','owner_id'])

# # # Load date into the legal table
#legal_csv_path = 'legal_dimension.csv'
#load_data(legal_csv_path, 'zipco.legal_dim',['legalDescription','subdivision', 'legal_id'])

# ##  Load data for the features table
#features_csv_path = 'features_dimension.csv'
#load_data(features_csv_path, 'zipco.features_dim',['features', 'propertyType', 'zoning','features_id'])


# # Load data for the location table
#location_csv_path = 'location_dimension.csv'
#load_data(location_csv_path, 'zipco.location_dim', ['county', 'zipCode', 'formattedAddress', 'state', 'city','location_id'])

# # -- Load data for the date table
#date_csv_path = 'date_dimension.csv'
#load_data(date_csv_path, 'zipco.date_dim',['lastSaleDate','year','month','monthName','quarter','date_id'])

#Load data for the fact table
#fact_columns = ['id', 'date_id','owner_id', 'features_id', 'location_id', 'bedrooms', 'squareFootage', 'taxAssessment2021_Values','propertyTaxes2021_Total',
                #'taxAssessment2022_Values','propertyTaxes2022_Total', 'taxAssessment2023_Values', 'propertyTaxes2023_Total','bathrooms', 'lotSize', 'lastSalePrice', 'longitude', 'latitude']

fact_csv_path = 'property_fact.csv'
load_data(fact_csv_path, 'zipco.fact_table', fact_columns)


Database connection successful!
Data loaded successfully into zipco.fact_table.
